In [1]:
#pip install pandas dash

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)


# To find launchsites
launchsites = spacex_df['Launch Site'].unique().tolist()
launchingSites = []
launchingSites.append({'label': 'All Sites', 'value': 'All Sites'})
for site in launchsites:
    launchingSites.append({'label': site, 'value': site})

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)


                                dcc.Dropdown(id='site-dropdown',options=launchingSites,
                                             placeholder='Select a Launch Site here',
                                             searchable = True , value = 'All Sites'),
                                html.Br(),


                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),


                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
                                value=[min_payload, max_payload]),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
# TASK 2:
#https://plotly.com/python/pie-charts/


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def pie_chart(launch_site):
    if launch_site == 'All Sites':
        fig = px.pie(values=spacex_df.groupby('Launch Site')['class'].mean(),
                     names=spacex_df.groupby('Launch Site')['Launch Site'].first(),
                     title='Total Success Launches by Site')
    else:
        fig = px.pie(values=spacex_df[spacex_df['Launch Site']==str(launch_site)]['class'].value_counts(normalize=True),
                     names=spacex_df['class'].unique(),
                     title='Total Success Launches for Site {}'.format(launch_site))
    return(fig)


# TASK 4:
#https://plotly.com/python/line-and-scatter/

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider',component_property='value')])
def Scatter_payload_chart(launch_site, payload_mass):
    if launch_site == 'All Sites':
        fig = px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])],
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for All Sites')
    else:
        df = spacex_df[spacex_df['Launch Site']==str(launch_site)]
        fig = px.scatter(df[df['Payload Mass (kg)'].between(payload_mass[0], payload_mass[1])],
                x="Payload Mass (kg)",
                y="class",
                color="Booster Version Category",
                hover_data=['Launch Site'],
                title='Correlation Between Payload and Success for Site {}'.format(launch_site))
    return(fig)

if __name__ == '__main__':
    app.run_server(debug=False, mode='external', port=8050)
    
#To get the output in new tab copy below link search bar of web browser
# http://127.0.0.1:8050/ 

#In case above link not working then use host parameter
'''
if __name__ == '__main__':
    app.run_server(debug=False, mode='external', port=8050, host='127.0.0.1')
'''

C:\Users\HP\AppData\Local\Temp\ipykernel_14872\4250164936.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\HP\AppData\Local\Temp\ipykernel_14872\4250164936.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

# Finding Insights Visually

1. Which site has the largest successful launches?

Ans- <code>KSC LC-39A</code>

2. Which site has the highest launch success rate?

Ans- <code>CCAFS SLC-40</code> has the highest launch success rate <code>42.9%</code>.

3. Which payload range(s) has the highest launch success rate?

Ans- Payload Range between <code>3000-4000 (kg)</code> has the highest launch success rate with 7 succesful launch.

4. Which payload range(s) has the lowest launch success rate?

Ans- Payload Range between <code>6000-9000 (kg)</code> has no succesful launch.Payload Range between <code>1000-2000 (kg) and 9000-10000 (kg)</code> has only 1 succesful launch.

5. Which F9 Booster version (v1.0,v1.1,FT,B4,B5, etc.) has the highest launch success rate?

Ans- <code>F9 Booster version (FT)</code> has the highest launch success rate.